## Tensorflow with simple 3-layer neural network using feedforward/backprop. 

This type of neural network gives no explanation of time or order.  It's not optimal for image classification and is just a demo, using the example MNIST data set provided by google. For better image classification performance, see convolutional neural networks.

Credits to Magnus Erik Hvass Pedersen's excellent series of [TensorFlow tutorials](https://github.com/Hvass-Labs/TensorFlow-Tutorials)  and Harrison's wonderful series on [Practical Machine Learning with Python](https://pythonprogramming.net/machine-learning-tutorials/)


## Import Libraries & Load the Data

In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets(train_dir="data/mnist", one_hot = False)

print("\ntraining size: {}".format(mnist.train.num_examples))
print("test size: {}".format(mnist.test.num_examples)) # Unused valid. set.

print("first 3 labels of test set: {}".format(mnist.test.labels[0:3]))

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz

training size: 55000
test size: 10000
first 3 labels of test set: [7 2 1]


Initialize some variables and constants

In [7]:
n_classes = 10 # numbers 0 - 9 in the dataset

train_batch_size = 128 # manipulate weights by 128 features at a time

image_size_1d = 28 * 28 # input images are 28 x 28

# None indicates the first dimension (corresponding to batch size) can be any size
x = tf.placeholder(dtype='float', shape=[None, image_size_1d], name='x') # height, width
y_true = tf.placeholder(dtype='int32', name='y_true')

total_iters = 15 # the number of times we will run our optimization over the training set

Set up the graph.  Weights are not optimized here, but in training. 

In [8]:
def model(X):
    # number of hidden nodes each layer
    n_nodes_hl1 = 700
    n_nodes_hl2 = 700
    n_nodes_hl3 = 700
    
    hidden_layer_1 = {'weights': tf.Variable(tf.random_normal([image_size_1d, n_nodes_hl1])), 
                     'biases' : tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_layer_2 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])), 
                     'biases' : tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_layer_3 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])), 
                     'biases' : tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])), 
                     'biases' : tf.Variable(tf.random_normal([n_classes]))}
    
    # X * weights + biases
    l1 = tf.add(tf.matmul(X, hidden_layer_1['weights']), hidden_layer_1['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_layer_2['weights']), hidden_layer_2['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_layer_3['weights']), hidden_layer_3['biases'])
    l3 = tf.nn.relu(l3)
    
    predicted_class = tf.add(tf.matmul(l3, output_layer['weights']), output_layer['biases'])
    
    return predicted_class

Train and Optimize. We pass the training set to train_model and make a prediciton based on the output of the model. 

cross-entropy is a continuous function that is always positive. If the predicted y equals the true y, cross-entropy equals zero.  Cross-entropy is used in classification.  This measures how well the model works on each image individually. The softmax_cross_entropy_with_logits function makes the sum of the inputs equal to 1, normalizing the values

cost is what we seek to miminimize.  It is a scalar representing the average cross-entropy for the classification of all images.  

optimizer minimizes cost by manipulating the weights and biases in the model. The optimization algorithm used to accomplish this is gradient descent.  Note that in the initialization, it is only being added to the graph and not executed.

In [ ]:
def train_model(x):
    y_pred_class = model(x)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y_pred_class, labels=y_true)
    cost = tf.reduce_mean( cross_entropy)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(cost)

    with tf.Session() as sess:
        # initialize the weights and biases before optimization starts
        sess.run(tf.global_variables_initializer())
        
        # train the network
        for iter in range(1, total_iters+1):
            iter_loss = 0
            for _ in range(int(mnist.train.num_examples / train_batch_size)):
                # get the next batch of training examples. 
                # x_batch holds a batch of images
                # y_true_batch holds the true labels for x_batch
                x_batch, y_true_batch = mnist.train.next_batch(train_batch_size)
                
                # fd_train holds the batch in a dictionary with the named placeholders
                # in the tensorflow graph
                fd_train = {x: x_batch, y_true: y_true_batch}
                _, i_loss = sess.run([optimizer, cost], feed_dict=fd_train)

                iter_loss += i_loss   
            print('iter', iter, 'of', total_iters, 'loss: ', iter_loss)
    
        correct = tf.nn.in_top_k(y_pred_class, y_true, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

        print('\ntraining set accuracy: ', accuracy.eval({x: mnist.train.images, y_true: mnist.train.labels})) 
        print('test set accuracy: ', accuracy.eval({x: mnist.test.images, y_true: mnist.test.labels}))         


In [ ]:
train_model(x)

iter 1 of 15 loss:  2108923.88188
iter 2 of 15 loss:  81097.015522
iter 3 of 15 loss:  42694.2836113
iter 4 of 15 loss:  24688.701741
iter 5 of 15 loss:  16048.4512193
iter 6 of 15 loss:  10551.8143533
iter 7 of 15 loss:  7186.29605607
iter 8 of 15 loss:  4823.72324636
iter 9 of 15 loss:  3292.5080048
iter 10 of 15 loss:  2412.67792343
iter 11 of 15 loss:  1712.10381894
iter 12 of 15 loss:  1149.28318126
